# Analysing the Data

In this notebook, we'll enrich the filtered events from the previous notebook with additional information. We'll use a combination of techniques to enrich the events:

1. Topic modeling using a Large Language Model (LLM) to extract topics from the posts
2. Creating embeddings for semantic search using a transformer model
3. Storing the enriched events in Redis for querying



Embeddings are vector representations of text that capture semantic meaning. They allow us to perform semantic search, which is a search based on meaning rather than exact keyword matching. In this notebook, we'll create embeddings for posts and store them in Redis for later querying.


## Recreating helping functions

In [1]:
@file:DependsOn("redis.clients:jedis:6.0.0")
@file:DependsOn("org.springframework.ai:spring-ai-transformers:1.0.0-RC1")
@file:DependsOn("ai.djl.huggingface:tokenizers:0.33.0")
@file:DependsOn("org.springframework.ai:spring-ai-ollama:1.0.0-RC1")
@file:DependsOn("org.springframework.ai:spring-ai-ollama:1.0.0-RC1")

In [2]:
import redis.clients.jedis.JedisPooled

val jedis = JedisPooled()

In [35]:
import org.springframework.ai.embedding.EmbeddingModel
import org.springframework.ai.ollama.OllamaChatModel
import org.springframework.ai.ollama.api.OllamaApi
import org.springframework.ai.ollama.api.OllamaOptions
import org.springframework.ai.transformers.TransformersEmbeddingModel
import org.springframework.ai.chat.messages.SystemMessage
import org.springframework.ai.chat.messages.UserMessage
import org.springframework.ai.chat.prompt.Prompt
import redis.clients.jedis.bloom.BFReserveParams
import java.nio.ByteBuffer
import java.nio.ByteOrder
import java.lang.Float

fun createBloomFilter(name: String) {
    runCatching {
        jedis.bfReserve(name, 0.01, 1_000_000L, BFReserveParams().expansion(2))
    }.onFailure {
        println("Bloom filter already exists")
    }
}

fun deduplicate(bloomFilter: String, uri: String): Boolean {
    return if (jedis.bfExists(bloomFilter, uri)) false else true
}

fun ack(bloomFilter: String, uri: String) {
    jedis.bfAdd(bloomFilter, uri)
}

val embeddingModel = TransformersEmbeddingModel()
embeddingModel.afterPropertiesSet()

val ollamaApi = OllamaApi.builder()
    .baseUrl("http://localhost:11434")
    .build()

val ollamaOptions = OllamaOptions.builder().model("deepseek-coder-v2").build()

val ollamaChatModel =  OllamaChatModel.builder()
    .ollamaApi(ollamaApi)
    .defaultOptions(ollamaOptions)
    .build()

fun createEmbedding(input: String): ByteArray {
    val embedding = embeddingModel.embed(input)
    val embeddingBytes = ByteArray(Float.BYTES * embedding.size)
    ByteBuffer.wrap(embeddingBytes).order(ByteOrder.LITTLE_ENDIAN).asFloatBuffer().put(embedding)
    return embeddingBytes
}

val topicModelingPromt = """
You are a topic classifier specialized in politics. Given a post, extract only politics-related topics—both explicitly mentioned and reasonably implied.

If a post mentions a political figure, event, party, law, or movement, infer related political topics or domains.

For example, if the post mentions “Green New Deal”, you may infer topics like “climate policy”, “progressive politics”, and “US Congress”.

Avoid generic terms like “news”, “statement”, or “speech”.
Only return relevant political topics.

Also avoid overly narrow items such as specific bill numbers or individual quotes.

If the topic or a very similar is already in the provided list of existing topics, use the one from the list, otherwise, feel free to create a new one.

Format your response as comma separated values (ALWAYS, I MEAN IT):
"topic1, topic2, topic3"

Examples:

Post:
Climate change policy needs serious bipartisan commitment.
Output:
“Climate Policy, Bipartisanship, Environmental Politics”
⸻
Post:
Macron’s recent comments on NATO expansion are causing waves.
Output:
“Emmanuel Macron, NATO, Foreign Policy, European Politics”
⸻
Post:
Just watched a debate on universal basic income — fascinating stuff!
Output:
“Universal Basic Income, Economic Policy, Social Welfare”
⸻
Post:
The Supreme Court decision today is a major turning point.
Output:
“Supreme Court, Judicial System, Constitutional Law”
⸻
Post:
Alexandria Ocasio-Cortez is pushing for stronger climate legislation.
Output:
“Alexandria Ocasio-Cortez, Climate Policy, Progressive Politics, US Congress”
"""

fun topicModeling(post: String, existingTopics: String): String {
    // Build a chat message
    val messages = listOf(
        SystemMessage(topicModelingPromt),
        UserMessage("Existing topics: $existingTopics"),
        UserMessage("Post: $post")
    )

    val response = ollamaChatModel.call(Prompt(messages))
    return response.result.output.text ?: ""
}

fun extractTopics(text: String): List<String> {
    val existingTopics = jedis.smembers("topics")
    return topicModeling(text, existingTopics.joinToString(", "))
        .replace("\"", "")
        .replace("“", "")
        .replace("”", "")
        .split(",")
        .map { it.trim() }
}

In [4]:
import redis.clients.jedis.resps.StreamEntry
import redis.clients.jedis.search.Document

data class Event(
    val did: String,
    val rkey: String,
    val text: String,
    val timeUs: String,
    val operation: String,
    val uri: String,
    val parentUri: String,
    val rootUri: String,
    val langs: List<String>,
    val similarityScore: Double
) {
    companion object {
        fun fromMap(entry: StreamEntry): Event {
            return fromMap(entry.fields)
        }

        fun fromMap(document: Document): Event {
            val fields = document.properties.associate { entry ->  entry.key to entry.value.toString()}
            return fromMap(fields)
        }

        fun fromMap(fields: Map<String, String>): Event {
            return Event(
                did = fields["did"] ?: "",
                rkey = fields["rkey"] ?: "",
                text = fields["text"] ?: "",
                timeUs = fields["timeUs"] ?: "",
                operation = fields["operation"] ?: "",
                uri = fields["uri"] ?: "",
                parentUri = fields["parentUri"] ?: "",
                rootUri = fields["rootUri"] ?: "",
                langs = fields["langs"]?.replace("[", "")?.replace("]", "")?.split(", ") ?: emptyList(),
                similarityScore = fields["similarityScore"]?.toDouble() ?: 0.0
            )
        }
    }
}

In [5]:
%use ktor-client
%use serialization
%use coroutines

In [6]:
@Serializable
data class LoginResponse(
    @SerialName("accessJwt") val accessJwt: String,
    @SerialName("refreshJwt") val refreshJwt: String,
    @SerialName("handle") val handle: String,
    @SerialName("did") val did: String,
    @SerialName("didDoc") val didDoc: DidDoc?,
    @SerialName("email") val email: String?,
    @SerialName("emailConfirmed") val emailConfirmed: Boolean?,
    @SerialName("emailAuthFactor") val emailAuthFactor: Boolean?,
    @SerialName("active") val active: Boolean,
    @SerialName("status") val status: String? = null
)

@Serializable
data class DidDoc(
    @SerialName("id") val id: String?
)

In [7]:
import io.ktor.client.HttpClient
import io.ktor.client.engine.cio.CIO
import io.ktor.client.plugins.contentnegotiation.ContentNegotiation
import io.ktor.serialization.kotlinx.json.json

val client = HttpClient(CIO) {
    install(ContentNegotiation) {
        json(Json {
            ignoreUnknownKeys = true
        })
    }
}

val API_URL = "https://bsky.social/xrpc"

val USERNAME = "devbubble.bsky.social"
val PASSWORD = System.getenv("DEVBUBBLE_TOKEN")

In [8]:
import io.ktor.client.call.body
import io.ktor.client.request.HttpRequestBuilder
import io.ktor.client.request.headers
import io.ktor.client.request.post
import io.ktor.client.request.setBody
import io.ktor.http.ContentType
import io.ktor.http.HeadersBuilder
import io.ktor.http.HttpStatusCode
import io.ktor.http.contentType

suspend fun getAccessToken(): String {
    val response = client.post("$API_URL/com.atproto.server.createSession") {
        contentType(ContentType.Application.Json)
        setBody(
            mapOf(
                "identifier" to USERNAME,
                "password" to PASSWORD
            )
        )
    }

    return if (response.status == HttpStatusCode.OK) {
        val result: LoginResponse = response.body()
        jedis.set("mainDid", result.did)
        println("✅ Login successful. DID: ${result.did}")
        result.accessJwt
    } else {
        println("⚠️ Authentication failed: ${response.status}")
        ""
    }
}

In [10]:
lateinit var blueskyToken: String;
runBlocking {
    blueskyToken = getAccessToken()
}

✅ Login successful. DID: did:plc:qdwb7czl4gdbu5go25dza3vo


In [11]:
@Serializable
data class SearchResponse(
    @SerialName("cursor") val cursor: String? = null,
    @SerialName("hitsTotal") val hitsTotal: Int? = null,
    @SerialName("posts") val posts: List<Post>
)

@Serializable
data class Post(
    @SerialName("uri") val uri: String,
    @SerialName("cid") val cid: String,
    @SerialName("author") val author: Author,
    @SerialName("indexedAt") val indexedAt: String,
    @SerialName("record") val record: Record?,
    @SerialName("replyCount") val replyCount: Int? = null,
    @SerialName("repostCount") val repostCount: Int? = null,
    @SerialName("likeCount") val likeCount: Int? = null,
    @SerialName("quoteCount") val quoteCount: Int? = null,

    )

@Serializable
data class Author(
    @SerialName("did") val did: String,
    @SerialName("handle") val handle: String,
    @SerialName("displayName") val displayName: String? = null,
    @SerialName("avatar") val avatar: String? = null
)

@Serializable
data class Record(
    @SerialName("text") val text: String? = null,
    @SerialName("embed") val embed: Embed? = null,
    @SerialName("createdAt") val createdAt: String
)

@Serializable
data class Embed(
    @SerialName("images") val images: List<Image>? = null
)

@Serializable
data class Image(
    @SerialName("thumb") val thumb: String? = null, // Nullable to handle missing values
    @SerialName("fullsize") val fullsize: String? = null,
    @SerialName("alt") val alt: String? = null // Alt text is also optional
)

In [12]:
import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.client.call.*
import io.ktor.http.*

import java.time.Instant
import java.time.temporal.ChronoUnit

suspend fun searchPosts(sinceTime: String, tag: String): List<Post> {
    val allPosts = mutableListOf<Post>()
    var cursor: String? = null

    println("🔍 Searching posts with tag: $tag since: $sinceTime")
    do {
        val response: HttpResponse = client.get("$API_URL/app.bsky.feed.searchPosts") {
            headers {
                append("Authorization", "Bearer $blueskyToken")
            }
            parameter("q", tag)
            parameter("sort", "latest")
            parameter("limit", 100)
            parameter("since", sinceTime)
            if (cursor != null) {
                parameter("cursor", cursor)
            }
        }

        if (response.status == HttpStatusCode.OK) {
            val result: SearchResponse = response.body()
            val posts = result.posts
            println("✅ Retrieved ${posts.size} posts. Total so far: ${allPosts.size + posts.size}.")
            allPosts.addAll(posts)
            cursor = result.cursor
        } else {
            println("⚠️ Failed to fetch posts. Status: ${response.status}")
            println(response.bodyAsText())
            break
        }
    } while (cursor != null)

    println("🎉 Finished fetching posts. Total retrieved: ${allPosts.size}.")
    return allPosts
}

In [13]:
import kotlinx.coroutines.runBlocking

runBlocking {
    val sinceTime = Instant.now().minus(15, ChronoUnit.HOURS).toString()
    searchPosts(sinceTime, "java")
}

🔍 Searching posts with tag: java since: 2025-05-18T02:01:47.504557Z
✅ Retrieved 100 posts. Total so far: 100.
✅ Retrieved 65 posts. Total so far: 165.
🎉 Finished fetching posts. Total retrieved: 165.


[Post(uri=at://did:plc:tzo73w2q7mcw2ohhnmjexuji/app.bsky.feed.post/3lphjvken2522, cid=bafyreif5r2gy5fineqncwzgsznip3xb7dj4lfu6d4x5nn2in5xcgsjlkay, author=Author(did=did:plc:tzo73w2q7mcw2ohhnmjexuji, handle=akirby80.net, displayName=akirby80, avatar=https://cdn.bsky.app/img/avatar/plain/did:plc:tzo73w2q7mcw2ohhnmjexuji/bafkreiciglxlak5vqoat4zcjbxtlrhk3id5w3qjeyypsf34gz35bwqyqjq@jpeg), indexedAt=2025-05-18T17:01:23.122Z, record=Record(text=Minecraft is the best.

Seed: -161003686673699055

Found on Java Edition 1.21.5

#Minecraft, embed=Embed(images=[Image(thumb=null, fullsize=null, alt=), Image(thumb=null, fullsize=null, alt=)]), createdAt=2025-05-18T17:01:22.333Z), replyCount=0, repostCount=0, likeCount=0, quoteCount=0), Post(uri=at://did:plc:aayuzazp5ehghd2yjbnnr2nk/app.bsky.feed.post/3lphjhtjd3c2g, cid=bafyreiah4342dorow6mihlvpsio5lrpzgxpsmrhqgpwbwyd7yczsk7byli, author=Author(did=did:plc:aayuzazp5ehghd2yjbnnr2nk, handle=tina-bp.bsky.social, displayName=ティナ, avatar=https://cdn.bsky.ap

In [14]:
val trendingTopicsRoute = listOf(
    "What are the most mentioned topics?",
    "What's trending right now?",
    "What’s hot in the network",
    "Top topics?",
    "What are people talking about?",
    "What people are discussing?",
)

In [15]:
@file:DependsOn("org.springframework.ai:spring-ai-redis-store:1.0.0-RC1")

In [16]:
import org.springframework.ai.vectorstore.redis.RedisVectorStore
import org.springframework.ai.vectorstore.redis.RedisVectorStore.MetadataField
import redis.clients.jedis.search.Schema.FieldType
import org.springframework.ai.vectorstore.SearchRequest
import org.springframework.web.reactive.function.server.router

val redisVectorStore = RedisVectorStore.builder(jedis, embeddingModel)
    .indexName("routeIdx")
    .contentFieldName("text")
    .embeddingFieldName("textEmbedding")
    .metadataFields(
        MetadataField("route", FieldType.TEXT)
    )
    .prefix("route:")
    .initializeSchema(true)
    .vectorAlgorithm(RedisVectorStore.Algorithm.FLAT)
    .build()
redisVectorStore.afterPropertiesSet()

In [17]:
import org.springframework.ai.document.Document
import java.util.UUID

trendingTopicsRoute.forEach { text ->
    redisVectorStore.add(
        listOf(
            Document(
                UUID.randomUUID().toString(),
                text,
                mapOf(
                    "route" to "trending_topics",
                    "text" to text,
                )
            )
        )
    )
}

In [18]:
redisVectorStore.similaritySearch(
    SearchRequest.builder()
        .topK(1)
        .query("Hey Dev Bubble. What's trending today? Excited to hear the news!")
        .build()
)

[Document{id='3588d11a-b1bc-4767-8cf7-e40927384bc1', text='What's trending right now?', media='null', metadata={distance=0.16146064, vector_score=0.16146064, route=trending_topics}, score=0.8385393619537354}]

In [19]:
import redis.clients.jedis.search.FTSearchParams
import redis.clients.jedis.search.Query

val userQuery = "Hey DevBubble, what's trending today? Excited to hear the news!"
val userQueryClauses = userQuery.split(Regex("""[!?,.:;()"\[\]{}]+"""))
userQueryClauses.filter { it.isNotBlank() }.forEach { clause ->

    val result = redisVectorStore.similaritySearch(
        SearchRequest.builder()
            .topK(1)
            .query(clause)
            .build()
    )

    println("Evaluated clause: $clause")
    println("Route: " + result?.first()?.metadata?.get("route"))
    println("Matched text: " + result?.first()?.text)
    println("Similarity Score: " + result?.first()?.score)
    println()
}

Evaluated clause: Hey DevBubble
Route: trending_topics
Matched text: What people are talking about?
Similarity Score: 0.6013960838317871

Evaluated clause:  what's trending today
Route: trending_topics
Matched text: What's trending right now?
Similarity Score: 0.9528361558914185

Evaluated clause:  Excited to hear the news
Route: trending_topics
Matched text: What are people talking about today?
Similarity Score: 0.6495983004570007



In [106]:
import redis.clients.jedis.search.FTSearchParams
import redis.clients.jedis.search.Query

fun matchRoute(query: String): List<String> {
    val userQueryClauses = query.split(Regex("""[!?,.:;()"\[\]{}]+"""))
    return userQueryClauses.filter { it.isNotBlank() }.flatMap { clause ->
        val result = redisVectorStore.similaritySearch(
            SearchRequest.builder()
                .topK(1)
                .query(clause)
                .build()
        )

        result?.forEach {
            println(it?.metadata?.get("route") as String)
            println(it?.score)
        }

        result?.filter { (it?.score ?: 0.0) > 0.80 }?.map {
            it?.metadata?.get("route") as String
        } ?: emptyList()
    }
    return emptyList()
}

In [21]:
import org.springframework.ai.chat.messages.SystemMessage
import org.springframework.ai.chat.messages.UserMessage
import org.springframework.ai.chat.prompt.Prompt
import java.time.LocalDateTime

fun trendingTopics(): Set<String> {
    val currentMinute = LocalDateTime.now().withSecond(0).withNano(0).toString()
    return jedis.smembers("topics")
        .map { it to jedis.cmsQuery("topics-cms:$currentMinute", it).first() }
        .sortedByDescending { it.second }
        .take(10)
        .map { it.first }
        .toSet()
}

In [23]:
trendingTopics()

[Economic Policy, Donald Trump, , European Union, Social Movements, Donald Trump Presidency, Financial Markets, Political Engagement, Political Extremism, Populism]

In [144]:
fun processUserRequest(query: String): String {
    val routes = matchRoute(query)
    println(routes)

    val enrichedData = routes.map { routes ->
        when (routes) {
            "trending_topics" -> trendingTopics()
            "summarization" -> summarization(query)
            else -> ""
        }
    }

    val systemPrompt = "You are a bot that helps users analyse posts about politics. You may be given a data set to help you answer questions. Answer in a max od 300 chars. I MEAN IT. It's a TWEET. Don't write more than 300 chars. Respond in only ONE paragraph. Be as concise as possible"

    return ollamaChatModel.call(
        Prompt(
            SystemMessage(systemPrompt),
            SystemMessage("Enriching data: $enrichedData"),
            UserMessage("User query: $query")
        )
    ).result.output.text ?: ""
}

In [108]:
processUserRequest("What are people talking about?")

redis.clients.jedis.exceptions.JedisDataException: Error parsing vector similarity query: query vector blob size (1536) does not match index's expected size (4096).

In [87]:
val summarizationRoute = listOf(
    "What are people saying about {topics}?",
    "What’s the buzz around {topics}?",
    "Any chatter about {topics}?",
    "What are folks talking about regarding {topics}?",
    "What’s being said about {topics} lately?",
    "What have people been posting about {topics}?",
    "What's trending in conversations about {topics}?",
    "What’s the latest talk on {topics}?",
    "Any recent posts about {topics}?",
    "What's the sentiment around {topics}?",
    "What are people saying about {topic1} and {topic2}?",
    "What are folks talking about when it comes to {topic1}, {topic2}, or both?",
    "What’s being said about {topic1}, {topic2}, and others?",
    "Is there any discussion around {topic1} and {topic2}?",
    "How are people reacting to both {topic1} and {topic2}?",
    "What’s the conversation like around {topic1}, {topic2}, or related topics?",
    "Are {topic1} and {topic2} being discussed together?",
    "Any posts comparing {topic1} and {topic2}?",
    "What's trending when it comes to {topic1} and {topic2}?",
    "What are people saying about the relationship between {topic1} and {topic2}?"
)

In [88]:
import org.springframework.ai.document.Document
import java.util.UUID

summarizationRoute.forEach { text ->
    redisVectorStore.add(
        listOf(
            Document(
                UUID.randomUUID().toString(),
                text,
                mapOf(
                    "route" to "summarization",
                    "text" to text,
                )
            )
        )
    )
}

In [136]:
import org.springframework.ai.chat.messages.SystemMessage
import org.springframework.ai.chat.messages.UserMessage
import org.springframework.ai.chat.prompt.Prompt

fun summarization(userQuery: String): List<String> {
    val queryTopics = extractTopics(userQuery)
    println(queryTopics)

    return queryTopics.flatMap { topic ->
        val query = Query("@topics:{'$topic'}")
            .returnFields("text")
            .setSortBy("time_us", false)
            .dialect(2)
            .limit(0, 10)

        val result = jedis.ftSearch(
            "postIdx",
            query
        )

        result.documents.map {
            document -> document.get("text").toString()
        }
    }
}

In [137]:
summarization("What's being said about Trump and Angela Merkel?")

[Donald Trump, Angela Merkel, Foreign Policy, International Relations]


[Granted some of these problems are insoluble: On some issues, Kraft is so close to Wu there's barely any daylight between them, so if you like Wu, why vote for Kraft? This is something a good campaign manager should have asked him before he even ran.,  Trump’s Sitcom Administration https:// medium.com/illumination/trumps -sitcom-administration-e8956035bb76?utm_source=flipboard&utm_medium=activitypub Posted into ILLUMINATION on Medium @ illumination-on-medium-Digitalmehmet 

| Details | Interest | Feed |, I'm sure DJT will be sending paper towels. Oh and maybe ICE can come to the disaster area so Noem Dome can strut around in camos while her flying monkeys in masks arrest people ., For putting this egregious MONSTER back in office by choosing him to be the RePub candidate, not once but TWICE:
DESTROY THE GOP!!!, How does electing Trump help the Palestinians?, And while Comey took a picture of a pile of rocks that they claim was a call for assassination, nothing was said, and the Secret

In [67]:
processUserRequest("What are people saying about Trump or Angela Merkel?")

[]
[]


 To analyze what people are saying about Trump and Angela Merkel, I would first collect relevant posts that mention either of these figures. Then, I would categorize the content based on sentiment analysis (positive, negative, neutral) to understand public opinion. Additionally, I might look at topics discussed in relation to both leaders to gain broader insights into popular narratives.

In [105]:
matchRoute("What are folks talking about tpics and Angela Merkel?")

trending_topics
0.8401669263839722


[]

In [98]:
val embeddingModel = TransformersEmbeddingModel()
embeddingModel.setModelResource("file:model/bge-large-en-v1.5/model.onnx")
embeddingModel.setTokenizerResource("file:model/bge-large-en-v1.5/tokenizer.json")
embeddingModel.afterPropertiesSet()

In [100]:
fun createEmbedding(input: String): ByteArray {
    val embedding = embeddingModel.embed(input)
    val embeddingBytes = ByteArray(Float.BYTES * embedding.size)
    ByteBuffer.wrap(embeddingBytes).order(ByteOrder.LITTLE_ENDIAN).asFloatBuffer().put(embedding)
    return embeddingBytes
}

val redisVectorStore = RedisVectorStore.builder(jedis, embeddingModel)
    .indexName("routeIdx")
    .contentFieldName("text")
    .embeddingFieldName("textEmbedding")
    .metadataFields(
        MetadataField("route", FieldType.TEXT)
    )
    .prefix("route:")
    .initializeSchema(true)
    .vectorAlgorithm(RedisVectorStore.Algorithm.FLAT)
    .build()
redisVectorStore.afterPropertiesSet()

summarizationRoute.forEach { text ->
    redisVectorStore.add(
        listOf(
            org.springframework.ai.document.Document(
                UUID.randomUUID().toString(),
                text,
                mapOf(
                    "route" to "summarization",
                    "text" to text,
                )
            )
        )
    )
}

trendingTopicsRoute.forEach { text ->
    redisVectorStore.add(
        listOf(
            org.springframework.ai.document.Document(
                UUID.randomUUID().toString(),
                text,
                mapOf(
                    "route" to "trending_topics",
                    "text" to text,
                )
            )
        )
    )
}

In [115]:
matchRoute("What are folks saying regarding Trump and Angela Merkel?")

trending_topics
0.8226513266563416


[trending_topics]

In [148]:
processUserRequest("What have people been posting about Trump or Angela Merkel?")

summarization
0.8771777749061584
[summarization]
[Donald Trump, Angela Merkel, Foreign Policy, Political Figures]


 People have been expressing concerns about Trump's support for Israel based on personal gains and criticizing his administration without addressing Gaza. Some tweets highlight Biden's promise towards Israel being misguided, while others show disdain for the way Trump handles international relations. There are also posts discussing the role of intelligence officials in contradicting Trump's claims and general discontent with political figures like Scott Jennings.

In [149]:
processUserRequest("What are people talking about?")

trending_topics
0.9923163056373596
[trending_topics]


 People are discussing a variety of topics related to politics. They're concerned with economic policy and the Democratic Party. There seems to be interest in how policies impact society and American life, including discussions around advocacy for social movements and disaster relief. European Union matters also come up occasionally, suggesting an awareness that global political dynamics play a role in domestic issues. Some are even discussing corporate politics and its implications on broader economic landscapes. The overall sentiment leans towards engagement with the political process, be it through commentary or discussion of policy decisions.